### This is an Example for Building a Retrieval-Augmented Generation (RAG) App with Unstructured IO
### A tool for extracting information from unstructured data and serves as the foundation for a wide range of applications, including information retrieval, data mining, and natural language processing.



In [4]:
!pip install -qU "unstructured-ingest[pdf]" unstructured langchain langchain-community transformers accelerate bitsandbytes sentence-transformers 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.12.0 requires aiofiles<24.0,>=22.0, but you have aiofiles 24.1.0 which is incompatible.
pymilvus 2.5.3 requires grpcio<=1.67.1,>=1.49.1, but you have grpcio 1.69.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
import os

os.environ["UNSTRUCTURED_API_KEY"] = "" # Add your key here
os.environ["UNSTRUCTURED_URL"] ="https://api.unstructured.io/general/v0/general" # You can find the URL in your personalized dashboard